# Import Packages

In [2]:
from croston import croston
import sys
sys.path.append('..')

import matplotlib.pyplot as plt
import utils
import pandas as pd
import numpy as np
import math
import random
from isoweek import Week
import lightgbm as lgb
import pickle
from sklearn.metrics import mean_squared_error
from tqdm.notebook import tqdm
from IPython.display import clear_output

# Settings

In [3]:
target_variable = 'Turnover' # Either 'Turnover' or 'Sales_QTY'

forecast_horizon = 1

n_splits = 4 # Number of cross validation splits

# Import Sales Data

- We do not import the data of 2021 so that we get the same data again when we execute the notebook in the future.


- For this approach, we just need Week, Year, Site_ID, SKU, Sales_QTY

In [3]:
query = 'SELECT Week, Year, Site_ID, SKU, Sales_QTY,Turnover FROM `globus-datahub-dev.Verteiler_PoC.timeseries_sales_weekly` where Year >= 2015 and (Year <= 2019 OR (Year = 2020 AND Week <= 47))  order by sku, site_id, year, week'
df_raw = utils.bq_to_dataframe(
    query, verbose=True).pipe(utils.reduce_memory_usage)

********************
Query Duration:  0:02:39.741328
Mem. usage decreased to 122.58 Mb (64.6% reduction)


In [4]:
df = df_raw.copy()
df

,Week,Year,Site_ID,SKU,Sales_QTY,Turnover
0,1,2015,101,1014557,5.0,129.500000
1,2,2015,101,1014557,2.0,51.799999
2,3,2015,101,1014557,0.0,0.000000
3,4,2015,101,1014557,1.0,25.900000
4,5,2015,101,1014557,0.0,0.000000
...,...,...,...,...,...,...
7561049,47,2020,101,1289328500,4.0,56.090000
7561050,47,2020,101,1289328600,2.0,29.799999
7561051,47,2020,101,1289328700,5.0,32.869999
7561052,47,2020,101,1289328800,4.0,27.600000


In [5]:
# Get rid of negative Sales / Turnover
df.loc[:, 'Sales_QTY'] = df.Sales_QTY.clip(lower=0)
df.loc[:, 'Turnover'] = df.Turnover.clip(lower=0)


# Forecasting using Croston

In [4]:
df = pd.read_csv('C:/dev/verteiler/03_Forecasting_LightGBM/data.csv')
df

c:\dev\verteiler\venv5\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Week,Year,Date,Site_ID,SKU,Sales_QTY,Turnover,Price,Season_Year,Vendor_ID,...,Warengruppe_Desc,Subclass_Desc,Class_Desc,Web_Article,Web_Article_From,Web_Article_To,CENSHARE_CAT_1,CENSHARE_CAT_2,CENSHARE_CAT_3,Classification
0,1,2015,2015W1,101,1014557,5.0,129.50,25.90,9999,126656,...,Küche,Zerkleinern,Kuechenhelfer,X,2015-01-28,9999-12-31,Kuche,Kuchenmesser,wiegemesser,A
1,2,2015,2015W2,101,1014557,2.0,51.80,25.90,9999,126656,...,Küche,Zerkleinern,Kuechenhelfer,X,2015-01-28,9999-12-31,Kuche,Kuchenmesser,wiegemesser,A
2,3,2015,2015W3,101,1014557,0.0,0.00,25.90,9999,126656,...,Küche,Zerkleinern,Kuechenhelfer,X,2015-01-28,9999-12-31,Kuche,Kuchenmesser,wiegemesser,A
3,4,2015,2015W4,101,1014557,1.0,25.90,25.90,9999,126656,...,Küche,Zerkleinern,Kuechenhelfer,X,2015-01-28,9999-12-31,Kuche,Kuchenmesser,wiegemesser,A
4,5,2015,2015W5,101,1014557,0.0,0.00,25.90,9999,126656,...,Küche,Zerkleinern,Kuechenhelfer,X,2015-01-28,9999-12-31,Kuche,Kuchenmesser,wiegemesser,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7821434,47,2020,2020W47,101,1289328500,4.0,56.09,14.90,9999,948934,...,Wohnboutique,Wohnaccessoires,HAY,X,2021-02-25,9999-12-31,Ordnung & Aufbewahrung,Korbe & Boxen,korbe,C
7821435,47,2020,2020W47,101,1289328600,2.0,29.80,14.90,9999,948934,...,Wohnboutique,Wohnaccessoires,HAY,X,2021-02-25,9999-12-31,Ordnung & Aufbewahrung,Korbe & Boxen,korbe,C
7821436,47,2020,2020W47,101,1289328700,5.0,32.87,6.57,9999,948934,...,Wohnboutique,Wohnaccessoires,HAY,X,2021-02-25,9999-12-31,Ordnung & Aufbewahrung,Korbe & Boxen,korbe,C
7821437,47,2020,2020W47,101,1289328800,4.0,27.60,6.90,9999,948934,...,Wohnboutique,Wohnaccessoires,HAY,X,2021-02-25,9999-12-31,Ordnung & Aufbewahrung,Korbe & Boxen,korbe,C


In [5]:
# count number of timeseries
df[['Site_ID', 'SKU']].drop_duplicates().shape

(59269, 2)

In [6]:
def run_croston(ts_list: list,fh: int) -> list:
    """
    Forecast timeseries using Croson's Method.
    :param ts_list: timeseries to be forecasted (list)
    :param fh: forecast horizon (int)
    :return: predicted value without historacal timeries (list)
    """
    fit_pred = croston.fit_croston(ts_list, fh)
    return list(fit_pred['croston_forecast'])

In [7]:
rmse = []

splits = utils.get_splits(df, fh=forecast_horizon, n_splits=n_splits)

for week, year in tqdm(splits, desc='n Cross-Validation Split'):
    week=int(week)
    year=int(year)
    
    results_dict = {"Site_ID":[],"SKU":[],"Forecast":[]}
    error_log    = {"Site_ID":[],"SKU":[]};
    
    # Prepare data
    df_train = df.loc[((df.Year == (Week(year, week))[0]) & (df.Week <= (Week(year, week))[1])) | (df.Year < (Week(year, week))[0])]
    targets = df.loc[df.Year == (Week(year, week)+1)[0]].loc[df.Week == (Week(year, week)+1)[1]][['SKU','Site_ID', 'Year','Week']+[target_variable]]

    # Forecast
    for i, sku_df in tqdm(df_train.groupby(['Site_ID','SKU']), desc='Time Series'):
        try:
            yhat = run_croston(sku_df[target_variable].to_list(),1 )
        except:
            yhat=[0]
            error_log["Site_ID"].append(sku_df.Site_ID.values[0])
            error_log["SKU"].append(sku_df.SKU.values[0])

        results_dict["Site_ID"].append(sku_df.Site_ID.values[0])
        results_dict["SKU"].append(sku_df.SKU.values[0])
        results_dict["Forecast"].append(yhat[0])
        
        
    result_df = pd.DataFrame(results_dict)
    result_df = result_df.merge(targets, on=['SKU', 'Site_ID'])

    rmse.append(mean_squared_error(result_df.Forecast, result_df[target_variable], squared=False))
    
    # Clear Cell Output
    clear_output()

print('RMSE:\n', rmse)
    
print('\nCrossvalidated RMSE: ', np.mean(rmse))

RMSE:
 [81.63929941009482, 70.98587476372798, 65.23137440818711, 70.85295219892164]

Crossvalidated RMSE:  72.17737519523288


In [8]:
result_df

,Site_ID,SKU,Forecast,Year,Week,Turnover
0,101,1014557,39.714225,2020,44,29.9
1,101,1014824,3.928686,2020,44,0.0
2,101,1016525,72.394127,2020,44,0.0
3,101,1016800,38.389998,2020,44,46.9
4,101,1017246,8.500609,2020,44,30.0
...,...,...,...,...,...,...
56927,590,1288237700,0.000000,2020,44,17.7
56928,590,1288237800,0.000000,2020,44,13.8
56929,590,1288237900,0.000000,2020,44,37.9
56930,590,1288341800,0.000000,2020,44,0.0
